In [7]:
# Downloading/Installing Gensim
# pip install --upgrade gensim

     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     -------------------- ------------------- 30.7/60.4 kB 1.4 MB/s eta 0:00:01
     ---------------------------------------- 60.4/60.4 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/24.0 MB 2.4 MB/s eta 0:00:10
    --------------------------------------- 0.4/24.0 MB 4.8 MB/s eta 0:00:05
    --------------------------------------- 0.5/24.0 MB 4.2 MB/s eta 0:00:06
   - -------------------------------------- 0.8/24.0 MB 4.5 MB/s eta 0:00:06
   - -------------------------------------- 1.0/24.0 MB 4.8 MB/s eta 0:00:05
   - -------------------------------------- 1.1/24.0 MB 4.2 MB/s eta 0:00:06
   -- ------------------------------------- 1.3/24.0 MB 4.5 MB/s eta 0:00:06
   -- ------------------------------------- 1.5/24.0 MB 4.4 MB/s eta 0:00:06
   -- ------------------------------------- 1.7/24.0 MB 4.4 MB/s eta 0:00:06
   --

In [1]:
# Import necessary libraries
import gensim.downloader
word2vec_google = gensim.downloader.load('word2vec-google-news-300')

# Part 2

In [2]:
import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

[nltk_data] Downloading package punkt to C:\Users\Jun
[nltk_data]     Wei\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
train = train.drop(['label-fine'], axis=1)
test = test.drop(['label-fine'], axis=1)
display(train)
display(test)

,label-coarse,text
0,0,How did serfdom develop in and then leave Russ...
1,1,What films featured the character Popeye Doyle ?
2,0,How can I find a list of celebrities ' real na...
3,1,What fowl grabs the spotlight after the Chines...
4,2,What is the full form of .com ?
...,...,...
5447,1,What 's the shape of a camel 's spine ?
5448,1,What type of currency is used in China ?
5449,4,What is the temperature today ?
5450,4,What is the temperature for cooking ?


,label-coarse,text
0,4,How far is it from Denver to Aspen ?
1,5,"What county is Modesto , California in ?"
2,3,Who was Galileo ?
3,0,What is an atom ?
4,4,When did Hawaii become a state ?
...,...,...
495,3,Who was the 22nd President of the US ?
496,1,What is the money they use in Zambia ?
497,4,How many feet in a mile ?
498,1,What is the birthstone of October ?


In [5]:
# Print unique labels in train and test dataset
train_labels = train['label-coarse'].sort_values().unique()
test_labels = test['label-coarse'].sort_values().unique()
print(f"Train labels: {train_labels}")
print(f"Test labels: {test_labels}")


Train labels: [0 1 2 3 4 5]
Test labels: [0 1 2 3 4 5]


## Re-classifying classes of original dataset

In [6]:
# Combine labels 2 and 5 to form single class 'OTHERS', denoted by label 2, for train and test dataset
train_mask = ((train['label-coarse']==2) | (train['label-coarse']==5))
test_mask = ((test['label-coarse']==2) | (test['label-coarse']==5))
train.loc[train_mask, 'label-coarse'] = 2
test.loc[test_mask, 'label-coarse'] = 2

display(train)
display(test)

,label-coarse,text
0,0,How did serfdom develop in and then leave Russ...
1,1,What films featured the character Popeye Doyle ?
2,0,How can I find a list of celebrities ' real na...
3,1,What fowl grabs the spotlight after the Chines...
4,2,What is the full form of .com ?
...,...,...
5447,1,What 's the shape of a camel 's spine ?
5448,1,What type of currency is used in China ?
5449,4,What is the temperature today ?
5450,4,What is the temperature for cooking ?


,label-coarse,text
0,4,How far is it from Denver to Aspen ?
1,2,"What county is Modesto , California in ?"
2,3,Who was Galileo ?
3,0,What is an atom ?
4,4,When did Hawaii become a state ?
...,...,...
495,3,Who was the 22nd President of the US ?
496,1,What is the money they use in Zambia ?
497,4,How many feet in a mile ?
498,1,What is the birthstone of October ?


In [7]:
train_labels_cleaned = train['label-coarse'].sort_values().unique()
test_labels_cleaned = test['label-coarse'].sort_values().unique()
print(f"Train labels: {train_labels_cleaned}")
print(f"Test labels: {test_labels_cleaned}")

Train labels: [0 1 2 3 4]
Test labels: [0 1 2 3 4]


In [8]:
from sklearn.model_selection import train_test_split

print("Original train size:", train.shape)

train_set, dev_set = train_test_split(train, test_size = 500)

print("New train size:", train_set.shape)
print("New development set size:", dev_set.shape)
display(train_set)
display(dev_set)

Original train size: (5452, 2)
New train size: (4952, 2)
New development set size: (500, 2)


,label-coarse,text
4721,1,What Leon Uris novel dealt with the Russian ca...
1155,0,What was unusual about Alexandra 's appearance...
2033,1,What bowl game began as an East-West contest b...
1954,0,What are the Valdez Principles ?
2286,4,What was the date of CNN 's first broadcast ?
...,...,...
4368,4,How many mines can still be found in the Falkl...
3418,0,What is the difference between a college and a...
5075,4,How deep is a fathom ?
4708,2,What Japanese city was once called Edo ?


,label-coarse,text
3331,1,What were hairy bank notes in the fur trade ?
2971,3,Name the Ranger who was always after Yogi Bear .
1866,2,Where is the Mayo Clinic ?
609,3,What was the name of the 187s Irish terrorist ...
2065,1,What do you call the feeling of having experie...
...,...,...
3383,3,Who shared a New York City apartment with Roge...
4394,0,What is the origin of the word Marijuana ?
2211,4,How much salt is in the oceans ?
3846,0,How do you do a bibliography page ?


## Neural Network Implementation

### Input Pre-processing
Architecture: Split sentences into words --> Apply word2vec embeddings --> Map embeddings to indices (word indexing) --> Padding --> Convert to PyTorch tensor --> Create DataLoader to load data for model training

In [9]:
# Function to split sentences into words and convert the column into a list of list of words

def split_text_into_words(df, column_name):
   # Split the text into words
   temp_df = df.copy()
   temp_df[column_name] = temp_df[column_name].str.split()
   
   # Convert the column into a list
   word_list = temp_df[column_name].tolist()
   
   # Return the list
   return word_list

In [10]:
train_wordList = split_text_into_words(train_set, 'text')
dev_wordList = split_text_into_words(dev_set, 'text')
test_wordList = split_text_into_words(test, 'text')


In [11]:
train_wordList

[['What',
  'Leon',
  'Uris',
  'novel',
  'dealt',
  'with',
  'the',
  'Russian',
  'capture',
  'of',
  'Berlin',
  '?'],
 ['What',
  'was',
  'unusual',
  'about',
  'Alexandra',
  "'s",
  'appearance',
  'in',
  'Josie',
  'and',
  'the',
  'Pussycats',
  '?'],
 ['What',
  'bowl',
  'game',
  'began',
  'as',
  'an',
  'East-West',
  'contest',
  'between',
  'Michigan',
  'and',
  'Stanford',
  'in',
  '192',
  '?'],
 ['What', 'are', 'the', 'Valdez', 'Principles', '?'],
 ['What', 'was', 'the', 'date', 'of', 'CNN', "'s", 'first', 'broadcast', '?'],
 ['Who',
  'penned',
  ':',
  '``',
  'Neither',
  'a',
  'borrower',
  'nor',
  'a',
  'lender',
  'be',
  "''",
  '?'],
 ['How', 'do', 'you', 'throw', 'a', 'housewarming', 'party', '?'],
 ['What',
  'cereal',
  'goes',
  '``',
  'snap',
  ',',
  'crackle',
  ',',
  'pop',
  "''",
  '?'],
 ['Tell',
  'me',
  'what',
  'city',
  'the',
  'Kentucky',
  'Horse',
  'Park',
  'is',
  'near',
  '?'],
 ['How', 'many', 'miles', 'is', 'it', 'fr

In [12]:
# Function to convert list of sentences (list of list of words) into a matrix format
def createMatrix(sentences, word2Idx):

    dataset = []

    wordCount = 0
    unknownWordCount = 0

    for sentence in sentences:
        wordIndices = []

        for word in sentence:
            wordCount += 1
            if word in word2Idx:
                wordIdx = word2Idx[word]
            elif word.lower() in word2Idx:
                wordIdx = word2Idx[word.lower()]
            else:
                wordIdx = word2Idx['UNKNOWN_TOKEN']
                unknownWordCount += 1
            
            # Get the label and map to int
            wordIndices.append(wordIdx)

        dataset.append(wordIndices)

    return dataset

In [13]:
# Finding the longest sentence in the whole dataset (train, dev and test)
train_maxlen = train_set['text'].str.split().str.len().max()
dev_maxlen = dev_set['text'].str.split().str.len().max()
test_maxlen = test['text'].str.split().str.len().max()

print(train_maxlen, dev_maxlen, test_maxlen)

37 28 17


In [14]:
# Function to pad each sentence to a fixed length (max sentence length)
def padding(wordList):
    max_sentence_length = 37
    for sentence in wordList:
        num_zeros = max_sentence_length-len(sentence)
        sentence.extend([0]*num_zeros)
    return wordList


In [15]:
# Create label set and word set
labelSet = set()
wordSet = set()

# unique words and labels in data  
for dataset in [train_set, test, dev_set]:
    for index, row in dataset.iterrows():
        sentence = row['text']
        label = row['label-coarse']
        labelSet.add(label)
        for word in sentence.split():
            wordSet.add(word.lower())


embed_size = len(word2vec_google.vectors[0])

# Get all words in the word2vec model
w2v_dictionary = list(word2vec_google.key_to_index.keys())

wordEmbeddings = []
# adding vector for padding token
wordEmbeddings.append(np.zeros(embed_size))
# adding vector for unknown vector intialised with uniform distribution
wordEmbeddings.append(np.random.uniform(-0.25, 0.25, embed_size))

# Initialise dictionary to map each word to a unique index
word2Idx = {}

# add padding+unknown
word2Idx["PADDING_TOKEN"] = len(word2Idx)
word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)

# adding vector for words present in the CoNLL2003 dataset
for word in w2v_dictionary:
    try:
        if word.lower() in wordSet:
            embedding_vector = word2vec_google.get_vector(word)
            if embedding_vector is not None:
                wordEmbeddings.append(embedding_vector)
                word2Idx[word] = len(word2Idx)
    except Exception as e:
        pass

wordEmbeddings = np.array(wordEmbeddings)

# format: [[padded wordindices], [caseindices], [padded char indices], [label indices]]

In [16]:
# Converting list of word indices and labels into PyTorch tensors
trainSentences = torch.tensor(padding(createMatrix(train_wordList, word2Idx)))
devSentences = torch.tensor(padding(createMatrix(dev_wordList, word2Idx)))
testSentences = torch.tensor(padding(createMatrix(test_wordList, word2Idx)))

trainLabels = torch.tensor(train_set['label-coarse'].tolist())
devLabels = torch.tensor(dev_set['label-coarse'].tolist())
testLabels = torch.tensor(test['label-coarse'].tolist())

In [17]:
# Create DataLoader for batched model training

from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
   def __init__(self, data, labels):
       self.data = data
       self.labels = labels

   def __len__(self):
       return len(self.data)

   def __getitem__(self, idx):
       return self.data[idx], self.labels[idx]

# Create DataLoader
train_dataloader = DataLoader(CustomDataset(trainSentences, trainLabels), batch_size=64, shuffle=True)
dev_dataloader = DataLoader(CustomDataset(devSentences, devLabels), batch_size=64, shuffle=True)


In [18]:
trainSentences.dtype

torch.int64

In [19]:
# Early stop as regularization to prevent overfitting (stops model training when dev set stops improving)

class EarlyStopper:
    def __init__(self, patience=10, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.max_accuracy = -np.inf

    def early_stop(self, accuracy):
        if accuracy > self.max_accuracy:
            self.max_accuracy = accuracy
            self.counter = 0
        elif accuracy < (self.max_accuracy - self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [20]:
wordEmbeddings.shape

(22955, 300)

### BiLSTM Implementation

In [26]:
# Define your Bi-LSTM model
class BiLSTMModel(nn.Module):
    def __init__(self, hidden_size, num_classes,wordEmbeddings,aggregation_method, dropout_rate):
        super(BiLSTMModel, self).__init__()  # Call the superclass's __init__ method

        self.wordEmbeddings = wordEmbeddings
        self.hidden_size = hidden_size
        self.aggregation_method = aggregation_method
        self.dropout_rate = dropout_rate

        self.embedding = nn.Embedding.from_pretrained(embeddings=self.wordEmbeddings,freeze=True,padding_idx=0)
        self.dropout = nn.Dropout(p=self.dropout_rate)

        self.bilstm = nn.LSTM(input_size=self.wordEmbeddings.shape[1], hidden_size=self.hidden_size, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, num_classes)  # Multiply by 2 for bidirectional
        self.softmax = nn.Softmax(dim=1) 


    def forward(self, x):
        x = x.to(torch.long)  # Convert the data type of the inputs        
        embed_out = self.embedding(x)
        embed_out = embed_out.to(torch.float32)
        lstm_out, _ = self.bilstm(embed_out)

        # Create a mask that is the same size as the input tensor
        mask = (x != 0).float()
        # Apply the mask to the output of the LSTM layer
        lstm_out = lstm_out * mask.unsqueeze(-1)

        if self.aggregation_method == 'avg':
           agg_output = torch.sum(lstm_out, dim=1) / torch.sum(mask, dim=1).unsqueeze(-1)
        elif self.aggregation_method == 'max':
            agg_output = torch.max(lstm_out, dim=1)[0]

        fc_out = self.fc(agg_output)
        softmax_out = self.softmax(fc_out)
        
        return softmax_out

### Machine Learning Model training

In [22]:
# Creating function for training of machine learning model
def train(model, train_dataloader, dev_dataloader,criterion, optimizer, epochs, early_stopping):

   train_accs = []
   dev_accs = []

   for epoch in range(epochs):

      model.train()
      correct = 0
      total = 0      
      for inputs,labels in train_dataloader:

         # Zero the parameter gradients
         optimizer.zero_grad()

         # Forward pass
         outputs = model(inputs)
         loss = criterion(outputs, labels)

         preds = torch.argmax(outputs,dim=1)
         
         # Backward pass and optimize
         loss.backward()
         optimizer.step()

         # Calculate accuracy
         correct += torch.sum(preds == labels)
         total += labels.size(0)
      train_accuracy = correct.double()/total
      train_accs.append(train_accuracy)

      model.eval()
      correct = 0
      total = 0
      with torch.no_grad():
         for inputs,labels in dev_dataloader:

            # Forward pass
            outputs = model(inputs)

            preds = torch.argmax(outputs,dim=1)

            # Calculate accuracy
            correct += torch.sum(preds == labels)
            total += labels.size(0)

      dev_accuracy = correct.double()/total
      dev_accs.append(dev_accuracy)
      
      print(f"Epoch: {epoch+1}, Train Acc: {train_accuracy}, Dev Acc: {dev_accuracy}")

      if early_stopping.early_stop(dev_accuracy):
         print(f"Early Stopping at Epoch:{epoch+1}")
         break

   return model,train_accuracy,dev_accuracy

### Initialise model

#### Average pooling aggregation method

In [28]:
avg_pool_blstm = BiLSTMModel(hidden_size=256, num_classes=5, wordEmbeddings=torch.tensor(wordEmbeddings), aggregation_method='avg', dropout_rate=0.5)

early_stopper = EarlyStopper()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(avg_pool_blstm.parameters(), lr=0.001)

In [29]:
avg_pool_blstm, train_acc,val_acc = train(avg_pool_blstm, train_dataloader, dev_dataloader, criterion=loss_fn, early_stopping=early_stopper, epochs=15, optimizer=optimizer)

Epoch: 1, Train Acc: 0.4303311793214863, Dev Acc: 0.62
Epoch: 2, Train Acc: 0.6698303715670436, Dev Acc: 0.642
Epoch: 3, Train Acc: 0.7146607431340872, Dev Acc: 0.704
Epoch: 4, Train Acc: 0.779886914378029, Dev Acc: 0.802
Epoch: 5, Train Acc: 0.8061389337641357, Dev Acc: 0.802
Epoch: 6, Train Acc: 0.8174474959612278, Dev Acc: 0.83
Epoch: 7, Train Acc: 0.8376413570274637, Dev Acc: 0.81
Epoch: 8, Train Acc: 0.8515751211631664, Dev Acc: 0.83
Epoch: 9, Train Acc: 0.8784329563812601, Dev Acc: 0.828
Epoch: 10, Train Acc: 0.8560177705977383, Dev Acc: 0.844
Epoch: 11, Train Acc: 0.8907512116316639, Dev Acc: 0.836
Epoch: 12, Train Acc: 0.9067043618739903, Dev Acc: 0.836
Epoch: 13, Train Acc: 0.9075121163166397, Dev Acc: 0.852
Epoch: 14, Train Acc: 0.9155896607431341, Dev Acc: 0.84
Epoch: 15, Train Acc: 0.9137722132471728, Dev Acc: 0.846


#### Max pooling aggregation method

In [30]:
max_pool_blstm = BiLSTMModel(hidden_size=256, num_classes=5, wordEmbeddings=torch.tensor(wordEmbeddings), aggregation_method='max', dropout_rate=0.5)

early_stopper = EarlyStopper()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(max_pool_blstm.parameters(), lr=0.001)

In [31]:
max_pool_blstm, train_acc,val_acc = train(max_pool_blstm, train_dataloader, dev_dataloader, criterion=loss_fn, early_stopping=early_stopper, epochs=15, optimizer=optimizer)

Epoch: 1, Train Acc: 0.43820678513731826, Dev Acc: 0.7
Epoch: 2, Train Acc: 0.7033521809369951, Dev Acc: 0.77
Epoch: 3, Train Acc: 0.7881663974151858, Dev Acc: 0.806
Epoch: 4, Train Acc: 0.8241114701130856, Dev Acc: 0.808
Epoch: 5, Train Acc: 0.8491518578352181, Dev Acc: 0.808
Epoch: 6, Train Acc: 0.8852988691437803, Dev Acc: 0.832
Epoch: 7, Train Acc: 0.9089256865912763, Dev Acc: 0.828
Epoch: 8, Train Acc: 0.9206381260096931, Dev Acc: 0.836
Epoch: 9, Train Acc: 0.9206381260096931, Dev Acc: 0.828
Epoch: 10, Train Acc: 0.9333602584814217, Dev Acc: 0.844
Epoch: 11, Train Acc: 0.9432552504038773, Dev Acc: 0.846
Epoch: 12, Train Acc: 0.9523424878836834, Dev Acc: 0.852
Epoch: 13, Train Acc: 0.9579967689822294, Dev Acc: 0.862
Epoch: 14, Train Acc: 0.9634491114701131, Dev Acc: 0.87
Epoch: 15, Train Acc: 0.965064620355412, Dev Acc: 0.858


### Machine Learning Model evaluation
Perform ML model evaluation by running on test dataset and predicting the labels

In [55]:
test_dataloader = DataLoader(CustomDataset(testSentences, testLabels), batch_size=64, shuffle=True)

In [61]:
def test(model, test_dataloader):
   model.eval()
   correct = 0
   total = 0
   with torch.no_grad():
       for inputs, labels in test_dataloader:
           outputs = model(inputs)
           preds = torch.argmax(outputs, dim=1)
           correct += torch.sum(preds == labels)
           total += labels.size(0)
   test_accuracy = correct.double() / total
   return test_accuracy

In [62]:
test_accuracy = test(avg_pool_blstm, test_dataloader)
print(f"Test Accuracy: {test_accuracy}")

Test Accuracy: 0.9
